In [40]:
import os
import csv
import requests
import pandas as pd
import numpy as np
import configparser

In [97]:
config = configparser.ConfigParser()
config.read(".env")
APIKEY = config["RAPIDAPI"]["KEY"]

## Outline of Data Extraction Process

 ---
 
 Using the RapidApi, the data will be collected in the following order.
 
 * The symbols of market movers will be gotten using the endpoint "market/v2/get-movers" 
 * The market movers will be added as query to the endpoint "stock/v3/get-historical-data" to collect historical data of the tickers
 * The symbols of the market movers will also be used as query to get their summary which includes the financial earnings using the endpoint stock/v2/get-summary
 * Currency pairs selected for the project are used as query in the endpoint "market/get-spark" to collect data according to interval and range. 
 * The collected data will initially be saved in pandas dataframe. 
 

#### Extract 10 stock losers, gainers and most-active stocks in GB region, get their symbols 

In [76]:
url = "https://yh-finance.p.rapidapi.com/market/v2/get-movers"

querystring = {"region":"GB","lang":"en-GB","count":"10","start":"0"}

headers = {
	"X-RapidAPI-Key": APIKEY,
	"X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring).json()

In [77]:
# Get the gainers from quotes
gainer_quotes = response["finance"]["result"][0]["quotes"]
gainers = []

for quote in gainer_quotes:
    gainers.append(quote["symbol"]) 

In [78]:
# Get the losers
losers_quotes = response["finance"]["result"][1]["quotes"]
losers = []

for quote in losers_quotes:
    losers.append(quote["symbol"]) 

In [79]:
# Get the most actives
mostactives_quotes = response["finance"]["result"][2]["quotes"]
mostactives = []

for quote in mostactives_quotes:
    mostactives.append(quote["symbol"]) 

In [80]:
# Combine all into one DF
movers_df = pd.DataFrame(list(zip(gainers, losers, mostactives)), columns=['gainers', 'losers', 'mostactives'])

movers_df

,gainers,losers,mostactives
0,0VC5.L,GCG.L,3ROK.L
1,3LAA.L,MH65.L,7DIG.L
2,FUL.L,0V6Y.L,KOD.L
3,KDR.L,0OI0.L,0VRF.L
4,GST.L,0O0J.IL,ICON.L
5,FUL.AQ,0MDB.L,0RQY.L
6,0HFT.L,3SMI.L,PREM.L
7,GUN.AQ,TUIB.L,GST.L
8,NTBR.L,FIRE.AQ,TYM.L
9,B90.L,VKCO.IL,BOIL.L


#### Extract historical data with the symbols 

In [93]:
def history_data(path, movers, key):
    data = []
    url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"
    for symbol in movers_df[movers]: 
        querystring = {"symbol": symbol,"region":"GB"}

        headers = {
            "X-RapidAPI-Key": key,
            "X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring).json()
        names = ['date', 'open', 'high', 'low', 'close', 'volume', 'adjclose', 'symbol', 'movers']
        for stock in response["prices"]:
            if stock['open']:
                history_data = {'date': stock['date'], 
                                    'open': stock['open'], 
                                   'high': stock['high'], 
                                   'low': stock['low'],
                                   'close': stock['close'], 'volume': stock['volume'], 
                                   'adjclose': stock['adjclose'], 'symbol': symbol,
                                   'movers': movers}
                data.append(history_data)

    with open(path, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=names)
        writer.writeheader()
        writer.writerows(data)

In [94]:
# Collect historical data of each gainer stock
gainers_history = history_data("data/gainers_history.csv", "gainers", APIKEY)
gainers_history

In [95]:
# Collect historical data of each loser stock
losers_history = history_data("data/losers_history.csv", "losers", APIKEY)
losers_history

In [96]:
# Collect historical data of each mostactive stock
actives_history = history_data("data/actives_history.csv", "mostactives", APIKEY)
actives_history

#### Extract Summary and Financial information of each stock

In [99]:
url = "https://yh-finance.p.rapidapi.com/stock/v2/get-summary"

querystring = {"symbol":"AMRN","region":"US"}

headers = {
	"X-RapidAPI-Key": APIKEY,
	"X-RapidAPI-Host": "yh-finance.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring).json()

In [ ]:
# ToDo
"""
  - Get the summaryProfile of stocks
  - Get the yearly earnings and revenue of all stocks
  - Get the dividend of all stock if any
"""

## Outline of Data Cleaning Process

---

  The collected data will here be cleaned and processed
  
  

## Data Wrangling and Transformation

## Data Storage and Loading